<a href="https://colab.research.google.com/github/hyunBBB/OMG/blob/main/13_googLeNet_implementation_on_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Activation, add, Flatten, AveragePooling2D, concatenate
from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from keras.utils import np_utils

width = 32
height = 32

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


https://www.tensorflow.org/api_docs/python/tf/keras/applications/InceptionV3

1. label 을 one-hot vector 로 변환

In [2]:
num_classes = 10
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)



2. list slicing 으로  train image / validation image 분리 

In [ ]:
validation_images, validation_labels = x_train[:500], y_train[:500]
train_images, train_labels =
# 못썼어...

In [ ]:
train_images.shape

(49500, 32, 32, 3)

In [ ]:
train_labels.shape

(49500, 10)

3. Inception module 정의

In [ ]:
def inception(x, filters): 
    pre_layer = x
    
    f1,f2,f3,f4 = filters

    # 1x1
      conv1 = Conv2d(f1, kernel_size=(1,1), padding='same', activation = 'relu')(pre_layer)
    # 1x1 & 3x3
      conv2 = Conv2d(f4, kernel_size=(1,1), padding='same', activation = 'relu')(pre_layer)
      conv2 = Conv2d(f2, kernel_size=(3,3), padding='same', activation = 'relu'(conv2))


    # 1x1 & 5x5
      conv2 = Conv2d(f4, kernel_size=(1,1), padding='same', activation = 'relu')(pre_layer)
      conv3 = Conv2d(f3, kernel_size=(5,5), padding='same', activation = 'relu'(conv3))

    # pooling & 1x1
      max_pool = MaxPooling2D(poo_size=(3,3), strides=(1,1), padding='same')(pre_layer)
      max_pool = Conv2D(f4, kernel_size=(1,1), padding = 'same')(max_pool)
      
    # output = [None(batch_size), w,h,c], c 기준 concatenate (axis = -1 )
    concat = concatenate([conv1, conv2, conv3, max_pool], axis=-1)
    
    return concat
    

4. GoogleNet (Incepceion V1) 모델 정의

In [1]:
from keras.layers.pooling import AveragePooling1D
input_shape =  x_train[0].shape
inputs = Input(shape= input_shape )

# conv랑 batch 사이에 max pooling 들어가야 하나, cifar 데이터에선 크기 너무 줄어들어서 뺐음
x = Conv2D(64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(inputs)
x = BatchNormalization()(x)
x = Conv2D(192, kernel_size=(3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x) 

#inception (input, [#1x1conv, #3x3conv, #5x5conv, #1x1reduction] )

# inception 3a
x = inception(X, [64, 128, 32, 32])

# inception 3b
x = inception(X, [128, 192, 96, 64])
X = MaxPooling2D(pool_size=(3,3), strides = (2, 2), padding='same')(X)



# inception 4a
x = inception(X, [192, 208, 48, 64])


#aux1

aux1 = AveragePooling2D(pool_size=(5,5), strides = (3,3), padding='valid')(X)
aux1 = Conv2D(128, kernel_size = (1,1), padding = 'same', activations='relu')(aux1)
aux1 = Flatten()(aux1)
aux1 = Dense(512, activation='relu')(aux1)
aux1 = Dense(10, activation = 'softmax')(aux1)


# inception 4b
X = inception(X, [160, 224, 64, 64])
# inception 4c
X = inception(X, [128, 256, 64, 64])

# inception 4d
X = inception(x, [112, 288, 64, 64])

#aux2
aux2 = AveragePooling2D(pool_size=(5,5), strides = (3,3), padding='valid')(X)
aux2 = Conv2D(128, kernel_size = (1,1), padding = 'same', activations='relu')(aux2)
aux2 = Flatten()(aux2)
aux2 = Dense(832, activation='relu')(aux2)
aux2 = Dense(10, activation = 'softmax')(aux2)



# inception 4e
x = inception(x, [256, 320, 128, 128])

# inception 5a
x = inception(x, [256, 320, 128, 128])
# inception 5b
x = inception(x, [384, 384, 128, 128 ])

x = AveragePooling2D(pool_size = (4,4), padding='valid')(x)
x = Dropout(0, 4)(x)


x=  Flatten()(x)

outputs = Dense(10, activation='softmax')(x)



NameError: ignored

In [ ]:
model = Model(Inputs = inputs, outputs = [aux1, aux2, outputs] )


# adam optimizer, categorical crossentropy, loss weight ratio (0.3,0.3,1.0), metric: accuracy
model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', loss_weight=[0.3,] ) 


In [ ]:
model.summary()

4. ImageDagaGenerator 로 normalization 및 augmentation 적용 (shear , zoom , horizontal flip 적용 )

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1,/255,
                                   shear_range = 0.2,
                                   zomm_range = 0.2,
                                   horizontal_filp = True)
validation_datagen = ImageDataGenerator(rescale = 1,/255,
                                   shear_range = 0.2,
                                   zomm_range = 0.2,
                                   horizontal_filp = True)

5. batch 32, epoch 10 으로 학습

In [ ]:
history = model.fit(train_datagen.flow(train_images, train_labels, batch_size = 32),
                    validation_data = validation_datagen.flow(validation_images, validation_labels, validation))

# 이쪽 다시 써야함.... 헝


6. train data, validation data 각각에 대해 마지막 layer의 loss 와 accuracy 

---



In [ ]:
import matplotlib.pyplot as plt

acc = history.history['']
val_acc = history.history['']

loss =history.history['']
val_loss = history.history['']

epochs_range = range(10)


# accuracy
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(label='Training Accuracy')
plt.plot( label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# loss
plt.subplot(1, 2, 2)
plt.plot(  label='Training Loss')
plt.plot(   label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()